<a href="https://colab.research.google.com/github/Riccardo-Venturi/Tesi_Script_Colab/blob/main/sequential_finding_of_forces_from_time_series__geometry_hu_moments_and_geometry_factors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#METRICHE
<details>
    Metriche di "Grandezza" (Quanto è grosso il danno?):

        AreaDelaminata: L'area totale dei pixel di classe "danno". Fondamentale.

        Dmax: Il diametro massimo che circoscrive tutto il danno.

        Damage-to-Hole Area Ratio (DHAR): Come hai detto tu, Area_Danno / Area_Foro. Semplice, adimensionale, potente. Ci dice se il danno è trascurabile o esteso.

    Metriche di "Forma" (Com'è fatto il danno?):

        Momenti di Hu: I tuoi 7 assi nella manica. Catturano l'essenza della forma in modo invariante. Essenziali per distinguere un danno compatto da uno fibroso.

        Elongation / Aspect Ratio: Come hai scritto, indica la direzionalità. Un valore alto suggerisce che il danno si sta propagando lungo una direzione di fibra preferenziale, che è un indicatore di pericolo.

        Solidity / Circularity: Metriche che hai già identificato. Ci dicono quanto il danno è "frastagliato" vs "compatto". Un danno a bassa solidità (molto frastagliato) ha più "punte" che agiscono da concentratori di stress, aumentando il rischio di propagazione delle cricche.

    Metriche "Radiali" (Come si distribuisce il danno attorno al foro?):

        Centroid Offset: Se il baricentro del danno è molto spostato rispetto al centro del foro, indica un danno asimmetrico e, di nuovo, una concentrazione di stress non uniforme.

        Copertura Angolare (Coverage): Delle 360° circonferenza del foro, quale percentuale è affetta da danno? Questo distingue un piccolo danno localizzato (bassa copertura) da una delaminazione perimetrale completa (alta copertura).

QUesta nuova cella dimostra che lgbm ha overfittato, quindi si preferisce prendere il modello MLP come utput con dati_forze_imputate

In [ ]:
!pip install -q lightgbm pandas matplotlib keras tensorflow scikit-learn
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt

# --- 1. CARICAMENTO E PREPARAZIONE DATASET ---
FILE_INPUT = 'LSTM_master_dataset.csv'

try:
    data = pd.read_csv(FILE_INPUT)
    print(f"Dataset '{FILE_INPUT}' caricato. Shape: {data.shape}")
except FileNotFoundError:
    print(f"ERRORE: File '{FILE_INPUT}' non trovato.")
    exit()

# Selezione delle feature rilevanti per la sequenza.
features_to_use = [
    'Forza_N', 'AreaDelaminata_mm2', 'Dmax_mm', 'DF_diametro',
    'DHAR_Area_Ratio', 'ShapeFactor', 'Hu_1', 'Hu_2', 'Hu_3', 'Hu_4', 'Hu_5', 'Hu_6', 'Hu_7'
]
target_features = ['AreaDelaminata_mm2', 'Dmax_mm', 'DF_diametro']

# Verifica che tutte le feature esistano nel DataFrame e crea il subset
features_to_use = [f for f in features_to_use if f in data.columns]
data_subset = data[features_to_use].astype('float32')

# --- 2. NORMALIZZAZIONE E CREAZIONE SEQUENZE ---
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data_subset)

# --- CORREZIONE: Ottieni gli indici delle colonne target PRIMA di creare le sequenze ---
all_columns = list(data_subset.columns)
target_indices = [all_columns.index(feature) for feature in target_features]


# Funzione aggiornata per lavorare solo con array NumPy e indici
def create_sequences(input_data, n_steps_in, target_col_indices):
    X, y = [], []
    for i in range(len(input_data) - n_steps_in):
        # La sequenza di input usa tutte le feature
        X.append(input_data[i:(i + n_steps_in), :])
        # L'output (y) prende solo le colonne target al passo temporale successivo
        y.append(input_data[i + n_steps_in, target_col_indices])
    return np.array(X), np.array(y)


n_steps = 15
X, y = create_sequences(scaled_data, n_steps, target_indices)


# --- 3. SUDDIVISIONE CRONOLOGICA TRAIN/TEST ---
test_size = 150
train_size = X.shape[0] - test_size

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(f"\nShape X_train: {X_train.shape}")
print(f"Shape y_train: {y_train.shape}")
print(f"Shape X_test: {X_test.shape}")
print(f"Shape y_test: {y_test.shape}")


# --- 4. COSTRUZIONE E ADDESTRAMENTO MODELLO LSTM ---
print("\nCostruzione e addestramento del modello LSTM...")
model = Sequential([
    LSTM(units=100, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    Dropout(0.2),
    LSTM(units=50, activation='relu'),
    Dropout(0.2),
    Dense(units=y_train.shape[1])
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    verbose=1
)
print("Addestramento completato.")


# --- 5. VALUTAZIONE MODELLO ---
# Predizioni
y_pred_scaled = model.predict(X_test)

# Inversione della normalizzazione
# Creiamo un array "dummy" con la stessa shape dei dati originali scalati
num_total_features = scaled_data.shape[1]

# Per le predizioni
dummy_pred = np.zeros((len(y_pred_scaled), num_total_features))
dummy_pred[:, target_indices] = y_pred_scaled
y_pred = scaler.inverse_transform(dummy_pred)[:, target_indices]

# Per i valori reali del test set
dummy_test = np.zeros((len(y_test), num_total_features))
dummy_test[:, target_indices] = y_test
y_test_orig = scaler.inverse_transform(dummy_test)[:, target_indices]


# --- 6. VISUALIZZAZIONE RISULTATI ---
print("\nGenerazione grafici di valutazione...")

# Grafico Loss
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss del Modello (MSE)')
plt.xlabel('Epoca'); plt.ylabel('Loss'); plt.legend(); plt.grid(True); plt.show()

# Grafici Predetto vs Reale
for i, feature_name in enumerate(target_features):
    plt.figure(figsize=(15, 5))
    plt.plot(y_test_orig[:, i], label='Valori Reali', color='blue', marker='.')
    plt.plot(y_pred[:, i], label='Valori Predetti', color='red', linestyle='--')
    plt.title(f'Predizione vs Valori Reali - {feature_name}')
    plt.xlabel('Time Step (nel test set)')
    plt.ylabel(feature_name)
    plt.legend(); plt.grid(True); plt.show()

In [ ]:
# --- CODICE AGGIUNTIVO PER L'INFERENZA SEQUENZIALE ---

# Impostazioni per l'inferenza
START_HOLE_NUMBER = 300
PREDICTION_HORIZON = 50 # Quanti passi futuri predire

# Trova l'indice nel DataFrame originale corrispondente al foro di partenza
start_index = data[data['NumeroForo'] == START_HOLE_NUMBER].index[0]

# Prendi l'ultima sequenza di dati reali disponibili come punto di partenza
# (dobbiamo prenderla dai dati scalati)
last_known_sequence = scaled_data[start_index - n_steps : start_index].reshape(1, n_steps, X.shape[2])

# Lista per salvare le predizioni fatte un passo alla volta
walk_forward_predictions_scaled = []

current_sequence = last_known_sequence.copy()

print(f"\nInizio inferenza Walk-Forward per {PREDICTION_HORIZON} passi dal foro {START_HOLE_NUMBER}...")

for _ in range(PREDICTION_HORIZON):
    # 1. Predici il prossimo passo
    next_step_pred_scaled = model.predict(current_sequence, verbose=0)
    walk_forward_predictions_scaled.append(next_step_pred_scaled.flatten())

    # 2. Crea un "passo temporale" completo per la prossima previsione.
    # Dobbiamo creare una riga con tutte le 13 feature.
    # Le feature target (es. Area, Dmax) vengono aggiornate con la predizione.
    # Le altre feature (es. Forza, Hu) devono essere "trasportate" dall'ultimo passo noto.
    # Per semplicità, in questa simulazione, replichiamo l'ultimo vettore di input e aggiorniamo solo le colonne predette.
    next_input_vector = current_sequence[0, -1, :].copy()
    next_input_vector[target_indices] = next_step_pred_scaled

    # 3. Aggiorna la sequenza: rimuovi il primo passo e aggiungi quello nuovo
    current_sequence = np.append(current_sequence[:, 1:, :], next_input_vector.reshape(1, 1, X.shape[2]), axis=1)

print("Inferenza completata.")


# --- Inversione della Normalizzazione per l'inferenza ---
walk_forward_predictions_scaled = np.array(walk_forward_predictions_scaled)

dummy_walk_forward = np.zeros((len(walk_forward_predictions_scaled), num_total_features))
dummy_walk_forward[:, target_indices] = walk_forward_predictions_scaled
y_pred_walk_forward = scaler.inverse_transform(dummy_walk_forward)[:, target_indices]


# --- Dati reali per confronto ---
real_data_for_comparison = data.iloc[start_index : start_index + PREDICTION_HORIZON]

# --- Grafico di confronto ---
print("Generazione grafico inferenza Walk-Forward...")
for i, feature_name in enumerate(target_features):
    plt.figure(figsize=(15, 5))
    # Dati reali
    plt.plot(real_data_for_comparison['NumeroForo'], real_data_for_comparison[feature_name], label='Valori Reali', color='blue', marker='.')
    # Predizioni sequenziali
    plt.plot(real_data_for_comparison['NumeroForo'], y_pred_walk_forward[:, i], label='Predizioni Walk-Forward', color='green', linestyle='--')
    plt.title(f'Inferenza Sequenziale vs. Reale dal Foro {START_HOLE_NUMBER} - {feature_name}')
    plt.xlabel('NumeroForo')
    plt.ylabel(feature_name)
    plt.legend(); plt.grid(True); plt.show()

#vecchio metodo con imputazione


In [ ]:
#@title questo predispone si sia fatto tutto il resto sotto come test; risulta un diario scientifico sperimentale
##prende il csv creato in precedenza con i valori mancanti delle forze e applica la lgbm che si è dimostrata quella con errore sperimentale
#più basso e migiore aderenza ai dati
# =============================================================================
# SNIPPET FINALE PER IMPUTAZIONE FORZE MANCANTI CON MODELLO OTTIMALE
# Metodo: LightGBM addestrato solo su 'NumeroForo'
# =============================================================================

import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. Caricamento del dataset con i buchi ---
try:
    df = pd.read_csv("/content/LSTM_master_dataset.csv")
    print("Dataset 'dataset_master_finalissimo.csv' caricato.")
except FileNotFoundError:
    print("ERRORE: Assicurati che il file 'dataset_master_finalissimo.csv' sia presente.")
    exit()

# --- 2. Preparazione Dati per l'Imputazione ---
# Converti colonna in numerico, gestendo i valori mancanti
df['Forza_N'] = pd.to_numeric(df['Forza_N'], errors='coerce')

# Separa i dati per addestrare il nostro modello di imputazione
df_known = df[df['Forza_N'].notna()].copy()

# Dati di addestramento: usiamo TUTTI i dati noti per addestrare il modello finale
X_train_final = df_known[['NumeroForo']]
y_train_final = df_known['Forza_N']

# Dati da imputare: le righe dove la Forza_N è mancante
X_to_impute = df[df['Forza_N'].isna()][['NumeroForo']]

# --- 3. Addestramento del Modello Vincente e Imputazione ---
print("\nAddestramento del modello di imputazione finale (LGBM-Base)...")
# Usiamo i parametri di default, che si sono dimostrati efficaci
imputation_model = lgb.LGBMRegressor(random_state=42, verbosity=-1)

# Addestriamo il modello su TUTTI i dati di forza disponibili
imputation_model.fit(X_train_final, y_train_final)

print("Predizione dei valori mancanti...")
# Eseguiamo la predizione
predicted_forces = imputation_model.predict(X_to_impute)

# --- 4. Creazione e Salvataggio del Dataset Definitivo ---
# Crea una copia del dataframe originale per non modificarlo
df_imputed_lgbm = df.copy()

# Riempi i valori NaN con le nostre predizioni
df_imputed_lgbm.loc[df_imputed_lgbm['Forza_N'].isna(), 'Forza_N'] = predicted_forces

# Salva il file CSV che userai per tutte le analisi future
output_filename = 'dataset_LGBM_imputato.csv'
df_imputed_lgbm.to_csv(output_filename, index=False)

print(f"\n✅ Dataset definitivo salvato come '{output_filename}'")
print("\nPrime righe con i valori imputati:")
display(df_imputed_lgbm.loc[df['Forza_N'].isna()].head())

# --- 5. Visualizzazione del Risultato (Highlight per la tesi) ---
print("\nGenerazione del grafico di confronto finale...")
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(20, 10))

# Dati reali originali
ax.scatter(df_known['NumeroForo'], df_known['Forza_N'],
           label='Dato Reale', s=15, c='royalblue', zorder=3)

# Dati che abbiamo appena imputato con il modello LGBM-Base
ax.scatter(X_to_impute['NumeroForo'], predicted_forces,
           label='Dato Imputato (LGBM-Base)', s=60, c='red', marker='X', zorder=5, edgecolor='black')

ax.set_title("Dataset Finale con Forze Imputate tramite Regressione su Usura", fontsize=18)
ax.set_xlabel("Numero del Foro (Sequenza Esperimento)", fontsize=14)
ax.set_ylabel("Forza (N)", fontsize=14)
ax.legend(fontsize=12, title="Stato del Dato")
plt.show()


In [ ]:
# Cella di confronto visuale rapido

df_mlp = pd.read_csv('dati_con_forze_imputate.csv')
df_lgbm = pd.read_csv('dataset_LGBM_imputato.csv')

# Estrai solo i valori imputati da entrambi
indici_mancanti = df_lgbm['NumeroForo'].isin(range(169, 177)) # Esempio per il primo blocco
forze_imputate_mlp = df_mlp[indici_mancanti]
forze_imputate_lgbm = df_lgbm[indici_mancanti]

# Grafico
plt.figure(figsize=(20, 10))
# Dati reali circostanti per contesto
plt.plot(df_lgbm['NumeroForo'], df_lgbm['Forza_N'], 'o', color='lightgray', label='Dati Reali')

# Dati imputati
plt.plot(forze_imputate_mlp['NumeroForo'], forze_imputate_mlp['Forza_N'], 's-r', label='Imputato con MLP (complesso)')
plt.plot(forze_imputate_lgbm['NumeroForo'], forze_imputate_lgbm['Forza_N'], 'X-g', label='Imputato con LGBM-Base (semplice/ottimale)')

plt.title('Confronto Visivo delle Imputazioni', fontsize=16)
plt.xlabel('NumeroForo')
plt.ylabel('Forza (N)')
plt.legend()
plt.xlim(160, 185) # Zoom sulla zona di interesse
plt.grid(True)
plt.show()

### **Tabella dei Dati Essenziali e Costanti del Progetto**

| Parametro / Dato | Valore | Unità | Fonte / Scopo | Note |
| :--- | :--- | :--- | :--- | :--- |
| **PARAMETRI FISICI & GEOMETRICI (Ground Truth)** | | | | |
| Diametro Nominale Foro (`D_nom`) | **6.0** | mm | Tesi / **Calibrazione** | La nostra "stele di Rosetta". È la misura reale che useremo per calibrare ogni patch. |
| Area Nominale Foro (`A_nom`) | **28.274** | mm² | Calcolata (`π*r²`) | Serve per calcolare `Adel` dai dati `Atot` umani e per lo `Shape Factor`. |
| Spessore Laminato | ~2.0 | mm | Tesi | Non usato nei modelli attuali, ma fondamentale per future simulazioni FEM. |
| **PARAMETRI DELLA SCANSIONE ORIGINALE** | | | | |
| Risoluzione Scansione | **600** | DPI | Tesi / Specifiche | Utilizzata per il calcolo della scala teorica. |
| Scala Teorica Originale | **23.62** | pixel/mm | Calcolata (`600/25.4`) | **Valida SOLO sulla scansione intera, non sulle patch ridimensionate.** |
| Diametro Foro in Pixel (Teorico) | **~142** | pixel | Calcolato (`6 * 23.62`) | Utile come controllo di sanità sui primi script di rilevamento. |
| **PARAMETRI DELLA PIPELINE ATTUALE (YOLO -> UNet)** | | | | |
| Dimensione Patch di Input (UNet) | **512 x 512** | pixel | Tuo script di crop | **Questo ridimensionamento ha causato la perdita della scala originale.** |
| **Scala Empirica Misurata su Patch** | **~35.8** | pixel/mm | Nostra misura manuale | **Valore indicativo** che conferma la discrepanza e la necessità di una calibrazione per patch. |
| **CLASSI NELLE MASCHERE UNet** | | | | |
| Sfondo | **0** | (valore pixel) | UNet script | Classe da ignorare nei calcoli. |
| Foro | **1** | (valore pixel) | UNet script | Maschera del foro. La UNet la separa dal danno. |
| Delaminazione | **2** | (valore pixel) | UNet script | Maschera del danno. **È questa che dobbiamo misurare.** |

---

### **Confronto Dati: Umano vs. Nostro (Il Problema da Risolvere)**

Questa tabella evidenzia la discrepanza che dobbiamo risolvere. Prendo il Foro 1 come esempio lampante:

| Metrica | Dato Umano (Tesi) | Dato Nostro (CSV sballato) | Stato |
| :--- | :--- | :--- | :--- |
| **Adel** | **3.08 mm²** | **6.14 mm²** | 🔴 **Sbagliato!** (Errore del ~100%) |
| **DMAX** | **6.22 mm** | **15.74 mm** | 🔴 **Sbagliato!** (Errore del ~150%) |
| **Forza** | **89.50 N** | (da predire) | 🔜 Obiettivo Futuro |

### **Cosa Facciamo Adesso (Il Piano Pratico Immediato)**

Come hai detto tu, lavoriamo ripartendo dalle patch e dalle maschere che hai già.

1.  **Azione #1: Calibrazione Automatica (ORA).**
    *   Prendiamo la cartella con le tue **patch radiografiche** (`.jpg`, 512x512).
    *   Eseguiamo lo **script di calibrazione automatica** che ti ho fornito. Questo script misurerà il diametro del foro in pixel in ogni patch e creerà il file `calibrazione_scale_patch.csv`.
    *   **Risultato:** Un file che associa ogni patch alla sua scala `px/mm` corretta.

2.  **Azione #2: Estrazione Features Corretta.**
    *   Modifichiamo lo script di estrazione features. Per ogni maschera:
        *   Legge il nome del file (es. `H001_..._pred.png`).
        *   Cerca la scala per `H001` nel file `calibrazione_scale_patch.csv`.
        *   Misura area e DMAX in pixel dalla maschera.
        *   **Usa la scala corretta per convertire i valori in mm e mm².**
    *   **Risultato:** Un nuovo file `features_corrette_e_verificate.csv` con dati finalmente confrontabili con quelli del tesista.

Iniziamo con l'**Azione #1**. Prepara la cartella con le patch `.jpg` e il codice di calibrazione che ti ho passato. È il passo più importante per sbloccare tutto il resto.


#### **Fase 1: La Scansione Originale (Il Mondo Reale in Pixel)**
<details>
*   **Immagine di Riferimento:** `T0_90_1A_ingresso.jpg` (e le altre scansioni complete).
*   **Scala Teorica (DPI):** 600 DPI (dots per inch).
    *   Conversione: 1 pollice = 25.4 mm
    *   **Scala Originale (px/mm):** `600 / 25.4 = **23.62 pixel per mm**`.
*   **Dimensione Fisica Riferimento:** Diametro nominale del foro `D_nom = **6.0 mm**`.
*   **Dimensione in Pixel (Teorica):** Sulla scansione originale, un foro da 6 mm dovrebbe avere un diametro di `6.0 mm * 23.62 px/mm ≈ **142 pixel**`.

#### **Fase 2: Dal Rilevamento YOLO al Ritaglio della Patch**

*   **Script di Riferimento:** `yolorilevazionefori.ipynb` e gli script successivi di ordinamento e crop.
*   **Processo:**
    1.  YOLO rileva i fori sulla scansione intera (`imgsz=1280` o `(H,W)` nativa).
    2.  Le coordinate delle bounding box vengono salvate e ordinate con K-Means (l'ordinamento bustrofedico).
    3.  Uno script di crop prende il centro `(cx, cy)` di ogni foro rilevato.
    4.  Viene ritagliata una patch quadrata attorno a quel centro. Hai usato `HS = 350` (patch 700x700) e `HS = 256` (patch 512x512).
    5.  **IL PASSAGGIO CRUCIALE:** In uno degli script, c'è `patch = cv2.resize(patch, (TARGET_SIZE, TARGET_SIZE), ...)` dove `TARGET_SIZE` è `512`.

#### **Fase 3: La Patch Radiografica (L'Input per la UNet)**

*   **Immagine di Riferimento:** L'immagine della patch singola che mi hai mostrato.
*   **Dimensione:** **512x512 pixel**.
*   **Il Problema:** Questa immagine è il risultato del `resize` della Fase 2. La sua scala in `pixel/mm` **NON è più 23.62**. È una nuova scala che dobbiamo calcolare.

#### **Fase 4: La Maschera Predetta (L'Output della UNet)**

*   **Immagine di Riferimento:** L'immagine della maschera in bianco e nero che mi hai mostrato.
*   **Dimensione:** **512x512 pixel** (la stessa della patch di input).
*   **Contenuto:** I valori dei pixel rappresentano le classi (es. 0=Sfondo, 1=Foro, 2=Danno).
*   **Obiettivo:** Misurare l'area e il DMAX del danno (pixel di classe 2) su questa maschera e convertirli in `mm²` e `mm` usando la scala corretta.


    *   **Mediana vs Media:** La tua intuizione sulla mediana è acuta. Se la distribuzione delle forze per un dato `NumeroForo` non è simmetrica, predire la media (come fa la MSE loss) potrebbe non essere la scelta migliore. I punti potrebbero essere "spiaccicati" (con una maggiore dispersione), e il nostro modello non lo cattura.


### Analisi più Rigorosa e Piano d'Azione Rivisto

Dato che l'obiettivo finale è definire un limite di danno accettabile, dobbiamo essere più critici. Invece di accettare ciecamente l'output della MLP, usiamolo come uno strumento esplorativo per rispondere a una domanda più profonda.

**Nuovo Obiettivo:** Stabilire se le *caratteristiche della forma del danno* (i momenti di Hu) contengono abbastanza informazione per distinguere tra un processo a "bassa forza" e uno ad "alta forza", al netto dell'effetto dominante dell'usura (`NumeroForo`).
<details>
Ecco un approccio più solido e un codice rivisto che si concentra su questa domanda.

#### Piano d'Azione (Versione 2.0 - più mirata)
1.  **Analisi di Correlazione:** Prima di addestrare un modello complesso, visualizziamo la correlazione tra le nostre features (soprattutto i momenti di Hu) e la `Forza_N`. Questo ci dirà quali descrittori sono più promettenti.
2.  **Feature Importance con un Modello Robusto:** Invece di una MLP (che è una "scatola nera"), usiamo un modello come **Gradient Boosting (es. LightGBM o XGBoost)**. Questi modelli ci forniscono una metrica diretta di **"Feature Importance"**, dicendoci quali variabili hanno usato di più per fare le loro predizioni. Questo è un passo diagnostico fondamentale.
3.  **Confronto Modelli:** Addestriamo due modelli:
    *   **Modello A (Baseline):** Predice la `Forza_N` usando **solo** `NumeroForo`. Questo ci dice qual è la performance che otteniamo basandoci solo sull'usura.
    *   **Modello B (Completo):** Predice la `Forza_N` usando `NumeroForo` + tutte le features del danno.
4.  **Valutazione:** Se il Modello B è **significativamente migliore** del Modello A, allora abbiamo la prova che **le forme del danno contengono informazione utile sulla forza, al di là del semplice effetto di usura.**


In [ ]:
### **Script di Calibrazione Automatica della Scala**
'''
Questo script fa una sola cosa, ma la fa bene:
1.  Legge ogni immagine `.jpg` da una cartella di input (le tue patch radiografiche).
2.  Usa l'algoritmo di visione artificiale `HoughCircles` per trovare il foro da 6 mm.
3.  Calcola la scala `pixel/mm` per quella specifica immagine.
4.  Salva tutti i risultati in un file CSV chiamato `calibrazione_scale_patch.csv`.

```python
'''# =============================================================================
# SCRIPT DI CALIBRAZIONE AUTOMATICA DELLA SCALA PER OGNI PATCH
# =============================================================================

# --- 1. INSTALLAZIONI E IMPORT NECESSARI ---
# (Esegui questa cella una sola volta all'inizio)
!pip install -q opencv-python pandas matplotlib
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from google.colab import files # Per scaricare il file CSV alla fine

# --- 2. IMPOSTAZIONI (LE UNICHE COSE DA MODIFICARE) ---

# Diametro reale del foro in millimetri. Questa è la nostra "verità" per la calibrazione.
DIAMETRO_REALE_FORO_MM = 6.0

# Imposta il percorso alla cartella che contiene le TUE patch radiografiche (.jpg).
# Esempio per Google Drive:
# from google.colab import drive
# drive.mount('/content/drive')
# RADIO_PATCHES_DIR = Path("/content/drive/MyDrive/PATCHES_RADIO_Dataset")

# Per un test iniziale, puoi caricare le immagini manualmente in una cartella in Colab.
RADIO_PATCHES_DIR = Path("/content/drive/MyDrive/PATCHES_RADIO_Dataset")#"/content/immagini_da_calibrare")
RADIO_PATCHES_DIR.mkdir(exist_ok=True) # Crea la cartella se non esiste
print(f"ATTENZIONE: Lo script cercherà le immagini in '{RADIO_PATCHES_DIR}'")
print("Carica le tue patch .jpg in quella cartella.")


# --- 3. FUNZIONE DI CALIBRAZIONE ---

def get_scale_from_radiograph(radiograph_path: Path, visualize=False):
    """
    Carica un'immagine radiografica, trova il foro centrale e calcola la scala pixel/mm.
    """
    if not radiograph_path.exists():
        print(f"Attenzione: Immagine non trovata a {radiograph_path}")
        return None, None

    # Carica l'immagine in scala di grigi
    image = cv2.imread(str(radiograph_path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        return None, None

    # Applica un leggero blur per ridurre il rumore e aiutare l'algoritmo
    blurred_image = cv2.GaussianBlur(image, (9, 9), 2)

    # Parametri per l'algoritmo HoughCircles. Potrebbero richiedere aggiustamenti.
    # dp: Rapporto inverso della risoluzione dell'accumulatore. 1.2 è un buon punto di partenza.
    # minDist: Distanza minima tra i centri dei cerchi. Mettiamola grande per trovare solo il foro principale.
    # param1: Soglia superiore per l'edge detector di Canny interno.
    # param2: Soglia per il centro del cerchio. Più è basso, più "falsi" cerchi trova.
    # minRadius, maxRadius: Limiti per il raggio del cerchio. FONDAMENTALI!

    # Stimiamo un range ragionevole per il raggio basandoci sulle dimensioni dell'immagine
    img_height, img_width = image.shape
    min_r = int(img_height * 0.15) # min raggio 15% dell'altezza
    max_r = int(img_height * 0.40) # max raggio 40% dell'altezza (più flessibile)

    circles = cv2.HoughCircles(blurred_image, cv2.HOUGH_GRADIENT, dp=1.2, minDist=img_width,
                               param1=70, param2=45, minRadius=min_r, maxRadius=max_r)

    if circles is not None:
        # Trovato almeno un cerchio!
        circles = np.round(circles[0, :]).astype("int")
        (x, y, r) = circles[0] # Prendiamo il primo (e unico) cerchio trovato

        diametro_pixel = r * 2
        scala_px_per_mm = diametro_pixel / DIAMETRO_REALE_FORO_MM

        if visualize:
            output_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
            cv2.circle(output_image, (x, y), r, (0, 255, 0), 4) # Cerchio verde
            cv2.circle(output_image, (x, y), 5, (0, 0, 255), -1) # Punto rosso al centro
            testo = f"Scala: {scala_px_per_mm:.2f} px/mm"
            cv2.putText(output_image, testo, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            return scala_px_per_mm, output_image

        return scala_px_per_mm, None
    else:
        # Nessun cerchio trovato automaticamente
        return None, image if visualize else None

# --- 4. BLOCCO DI ESECUZIONE ---

# Prendi tutte le immagini .jpg nella cartella
all_radio_paths = sorted(list(RADIO_PATCHES_DIR.glob("*.jpg")))
if not all_radio_paths:
    all_radio_paths = sorted(list(RADIO_PATCHES_DIR.glob("*.png"))) # Prova anche i png

if not all_radio_paths:
    print(f"❌ ERRORE: Nessuna immagine .jpg o .png trovata nella cartella '{RADIO_PATCHES_DIR}'.")
    print("Assicurati di aver caricato le immagini prima di eseguire questa cella.")
else:
    print(f"Trovate {len(all_radio_paths)} patch. Inizio calibrazione...")

    # Visualizziamo un esempio per un controllo di qualità
    print("\n--- TEST DI VISUALIZZAZIONE SULLA PRIMA IMMAGINE ---")
    scala_test, img_test = get_scale_from_radiograph(all_radio_paths[0], visualize=True)
    if scala_test:
        print(f"Scala calcolata per '{all_radio_paths[0].name}': {scala_test:.2f} px/mm")
        plt.figure(figsize=(6, 6))
        plt.imshow(cv2.cvtColor(img_test, cv2.COLOR_BGR2RGB))
        plt.title("Visualizzazione Rilevamento Cerchio")
        plt.axis('off')
        plt.show()
    else:
        print("Rilevamento fallito sull'immagine di test. Prova a modificare i parametri di HoughCircles (param2, minRadius, maxRadius).")

    # Eseguiamolo su tutte le immagini per creare il nostro file di calibrazione
    print("\n--- CALIBRAZIONE DI MASSA SULL'INTERO DATASET ---")
    calibration_data = []
    for path in tqdm(all_radio_paths, desc="Calibrazione Immagini"):
        scala, _ = get_scale_from_radiograph(path, visualize=False)

        entry = {'filename': path.name}
        if scala:
            entry['scala_px_per_mm'] = scala
        else:
            entry['scala_px_per_mm'] = np.nan # Mettiamo NaN se il rilevamento fallisce
        calibration_data.append(entry)

    # Converti i risultati in un DataFrame di Pandas
    df_calibrazione = pd.DataFrame(calibration_data)

    output_filename = "calibrazione_scale_patch.csv"
    df_calibrazione.to_csv(output_filename, index=False)

    print("\n--- RISULTATI CALIBRAZIONE ---")
    print(df_calibrazione.head())
    print(f"\n✅ File di calibrazione '{output_filename}' creato con successo!")

    # Controlliamo se ci sono stati fallimenti
    fallimenti = df_calibrazione['scala_px_per_mm'].isna().sum()
    if fallimenti > 0:
        print(f"\n⚠️ ATTENZIONE: {fallimenti} immagini non sono state calibrate correttamente.")
    else:
        print("\n🎉 OTTIMO: Tutte le immagini sono state calibrate con successo.")

    # Analizziamo la distribuzione delle scale
    plt.figure(figsize=(10, 5))
    df_calibrazione['scala_px_per_mm'].hist(bins=30, edgecolor='black')
    plt.title("Distribuzione delle Scale Calcolate (px/mm)")
    plt.xlabel("Scala (pixel/mm)")
    plt.ylabel("Numero di Patch")
    plt.show()

    # Offri il download del file CSV
    files.download(output_filename)

ora si applica alle maschere della unet

In [ ]:
# =============================================================================
# SCRIPT DI ESTRAZIONE FEATURES "INTELLIGENTE" CON SCALA CORRETTA
# =============================================================================

# --- 1. INSTALLAZIONI E IMPORT NECESSARI ---
!pip install -q opencv-python pandas
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
from google.colab import files

# --- 2. IMPOSTAZIONI (CONTROLLA QUESTI PERCORSI) ---

# Percorso al file CSV con le scale che hai appena generato
CALIBRATION_FILE_PATH = Path("/content/calibrazione_scale_patch.csv")

# Percorso alla cartella che contiene le TUE maschere predette dalla UNet (.png)
MASKS_DIR = Path("/content/drive/MyDrive/PATCHES_RADIO_Dataset")
MASKS_DIR.mkdir(exist_ok=True) # Crea la cartella se non esiste
print(f"ATTENZIONE: Lo script cercherà le maschere in '{MASKS_DIR}'")
print("Carica le tue maschere .png in quella cartella.")

# --- 3. CARICAMENTO DATI DI CALIBRAZIONE ---
try:
    df_calibrazione = pd.read_csv(CALIBRATION_FILE_PATH)
    # Creiamo un dizionario per una ricerca super veloce della scala
    # Assumiamo che il filename della maschera derivi da quello della patch
    # Es: H001_h001_..._pred.png -> H001_h001_....jpg
    # Quindi creiamo una chiave pulita
    def get_clean_key(filename):
        return filename.replace('.png', '.jpg')

    scale_dict = {row['filename']: row['scala_px_per_mm'] for _, row in df_calibrazione.iterrows()}
    print(f"✅ File di calibrazione caricato con successo. Contiene {len(scale_dict)} scale.")
except FileNotFoundError:
    print(f"❌ ERRORE: File di calibrazione non trovato in '{CALIBRATION_FILE_PATH}'. Assicurati che sia presente.")
    raise

# --- 4. FUNZIONE DI ESTRAZIONE FEATURES (AGGIORNATA) ---

def extract_calibrated_features(mask_path, scale_lookup):
    """
    Estrae le features da una maschera usando la scala di calibrazione corretta.
    """
    # Ricava il nome del file della radiografia originale per cercare la scala
    original_radio_filename = mask_path.name.replace('.png', '.jpg')

    # Prendi la scala corretta dal nostro dizionario
    scala = scale_lookup.get(original_radio_filename)

    if scala is None or pd.isna(scala):
        # Se non troviamo una scala per questa immagine, la saltiamo
        return None

    # Carica la maschera
    mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
    if mask is None:
        return None

    # Isola solo la maschera della delaminazione (classe 2)
    delam_mask = (mask == 2).astype(np.uint8)

    # --- Calcolo Area ---
    area_delam_px = cv2.countNonZero(delam_mask)
    area_delam_mm2 = area_delam_px / (scala ** 2)

    # --- Calcolo DMAX ---
    contours, _ = cv2.findContours(delam_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    d_max_px = 0
    if contours:
        # Unisci tutti i punti di tutti i contorni in un unico array
        all_points = np.concatenate(contours, axis=0)
        # Troviamo il rettangolo rotato con area minima che contiene tutti i punti
        rect = cv2.minAreaRect(all_points)
        # La diagonale di questo rettangolo è una buona stima di DMAX
        box = cv2.boxPoints(rect)
        dist1 = np.linalg.norm(box[0] - box[2])
        dist2 = np.linalg.norm(box[1] - box[3])
        d_max_px = max(dist1, dist2)

    d_max_mm = d_max_px / scala

    # (Opzionale, puoi aggiungere qui i Momenti di Hu o altri calcoli se servono)

    return {
        'FileMaschera': mask_path.name,
        'AreaDelaminata_mm2': area_delam_mm2,
        'Dmax_mm': d_max_mm,
        'Scala_Usata': scala
    }

# --- 5. BLOCCO DI ESECUZIONE ---
all_mask_paths = sorted({
    *MASKS_DIR.rglob("*.png"),
    *MASKS_DIR.rglob("*.jpg"),
    *MASKS_DIR.rglob("*.jpeg"),
})
if not all_mask_paths:
    print(f"❌ ERRORE: Nessuna maschera .png trovata in '{MASKS_DIR}'.")
else:
    print(f"\nTrovate {len(all_mask_paths)} maschere. Inizio estrazione features corrette...")

    features_list = []
    for path in tqdm(all_mask_paths, desc="Estrazione Features Calibrate"):
        feats = extract_calibrated_features(path, scale_dict)
        if feats:
            features_list.append(feats)

    df_features_corrette = pd.DataFrame(features_list)

    output_filename = "features_corrette_e_verificate.csv"
    df_features_corrette.to_csv(output_filename, index=False)

    print("\n--- RISULTATI ESTRAZIONE CORRETTA ---")
    print(df_features_corrette.head())
    print(f"\n✅ File con features corrette '{output_filename}' creato con successo!")

    # Offri il download
    files.download(output_filename)

Crezione del file csv dei dati di lavoro definitivo, mi fa merge di quelle maschere unet e del file di origine di Melis

In [ ]:
### **Notebook Definitivo: Creazione del Dataset Master Comparativo**

#### **CELLA 1: Setup, Caricamento Dati e Costanti**
'''*Questa cella imposta l'ambiente, definisce le costanti del nostro universo e carica tutti i file di input necessari.*

```python'''
# =============================================================================
# CELLA 1: SETUP, CARICAMENTO DATI E COSTANTI
# =============================================================================

# --- 1. Installazioni e Import ---
!pip install -q opencv-python pandas
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
import re
from google.colab import files

print("--- Ambiente Pronto ---")

# --- 2. Costanti Fisiche e di Progetto ---
D_NOMINALE_MM = 6.0
AREA_NOMINALE_MM2 = np.pi * (D_NOMINALE_MM / 2)**2
print(f"Costanti definite: D_nom = {D_NOMINALE_MM} mm, A_nom = {AREA_NOMINALE_MM2:.3f} mm^2")


# --- 3. Percorsi dei File di Input (CORRETTI) ---
CALIBRATION_FILE_PATH = Path("/content/calibrazione_scale_patch.csv")


# QUESTI SONO I LINK AI FILE "RAW" (Grezzi)
FORZA_URL_RAW = "https://github.com/Riccardo-Venturi/DatiBuchi/raw/main/forze%20T_0_90.xlsx"
GEOMETRIA_MELIS_URL_RAW = "https://github.com/Riccardo-Venturi/DatiBuchi/raw/main/Aree%20Delaminazioni%20T_0_90_rev1.xlsx"


# --- 4. Caricamento e Preparazione dei Dati di Input ---
try:
    # Dati di Calibrazione (da file locale)
    df_calibrazione = pd.read_csv(CALIBRATION_FILE_PATH)
    scale_dict = {row['filename']: row['scala_px_per_mm'] for _, row in df_calibrazione.iterrows()}
    print(f"✅ File di calibrazione caricato ({len(scale_dict)} voci).")

    # Dati di Forza di Maurizio (da URL raw usando pd.read_excel)
    df_forza_maurizio = pd.read_excel(FORZA_URL_RAW, engine='openpyxl',header=1)
    # Puliamo i dati, prendendo solo le colonne che ci servono
    df_forza_maurizio = df_forza_maurizio[['Foro', 'Fcorretta']].rename(columns={'Foro': 'NumeroForo', 'Fcorretta': 'Forza_N'})
    print(f"✅ File delle forze di Maurizio caricato ({len(df_forza_maurizio)} voci).")
#    # Dati Geometrici di Maurizio
#    df_geometria_maurizio = pd.read_excel(GEOMETRIA_MELIS_URL_RAW,engine='openpyxl')
#    print(f"✅ File di geometria di Maurizio caricato ({len(df_geometria_maurizio)} voci).")
#
except Exception as e:
    print(f"❌ ERRORE: Qualcosa è andato storto nel caricamento dei file. Dettagli: {e}")
    raise


In [ ]:
print(MASKS_DIR)

In [ ]:

# =============================================================================
# CELLA 3: ESTRAZIONE FEATURES GEOMETRICHE CALIBRATE (UNET)
# =============================================================================

def extract_hole_number(path):
    match = re.search(r'H(\d+)', path.stem)
    return int(match.group(1)) if match else -1

def extract_geometric_features(mask_path, scale_lookup):
    original_radio_filename = mask_path.name.replace('.png', '.jpg')
    scala = scale_lookup.get(original_radio_filename)
    if scala is None or pd.isna(scala): return None

    mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
    if mask is None: return None

    numero_foro = extract_hole_number(mask_path)
    delam_mask = (mask == 2).astype(np.uint8)

    # Calcoli geometrici
    area_delam_px = cv2.countNonZero(delam_mask)
    area_delam_mm2 = area_delam_px / (scala ** 2)

    d_max_mm = 0
    contours, _ = cv2.findContours(delam_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        all_points = np.concatenate(contours, axis=0)
        if len(all_points) > 0:
            rect = cv2.minAreaRect(all_points)
            box = cv2.boxPoints(rect)
            d_max_px = max(np.linalg.norm(box[0] - box[2]), np.linalg.norm(box[1] - box[3]))
            d_max_mm = d_max_px / scala

    # Feature ingegnerizzate
    df_diametro = d_max_mm / D_NOMINALE_MM
    area_cerchio_max_mm2 = np.pi * (d_max_mm / 2)**2
    denominatore_sf = area_cerchio_max_mm2 - AREA_NOMINALE_MM2
    shape_factor = area_delam_mm2 / denominatore_sf if denominatore_sf > 0 else 0

    return {
        'NumeroForo': numero_foro,
        'AreaDelaminata_unet': area_delam_mm2,
        'Dmax_unet': d_max_mm,
        'DF_unet': df_diametro,
        'ShapeFactor_unet': shape_factor,
        'FileMaschera': mask_path.name,
    }

# Eseguiamo l'estrazione
all_mask_paths = sorted({
    *MASKS_DIR.rglob("*.png"),
    *MASKS_DIR.rglob("*.jpg"),
    *MASKS_DIR.rglob("*.jpeg"),
})
if not all_mask_paths:
    print(f"❌ ERRORE: Nessuna maschera trovata in '{MASKS_DIR}'.")
else:
    geometric_features_list = []
    for path in tqdm(all_mask_paths, desc="Estraggo Geometria UNet"):
        feats = extract_geometric_features(path, scale_dict)
        if feats:
            geometric_features_list.append(feats)

    df_unet_geometric = pd.DataFrame(geometric_features_list)
    print("\n--- Dataset Geometrico (UNet) Calibrato ---")
    display(df_unet_geometric.head())

#### **CELLA 4: Calcolo Momenti di Hu (Cella Separata)**
'''*Come richiesto, una cella dedicata a calcolare i 7 Momenti di Hu, che sono ottimi descrittori di forma.*

```python'''
# =============================================================================
# CELLA 4: CALCOLO MOMENTI DI HU
# =============================================================================

def calculate_hu_moments(mask_path):
    mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
    if mask is None: return None

    numero_foro = extract_hole_number(mask_path)
    delam_mask = (mask == 2).astype(np.uint8)

    moments = cv2.moments(delam_mask)
    hu_moments = cv2.HuMoments(moments).flatten()
    hu_log = np.sign(hu_moments) * np.log10(np.abs(hu_moments) + 1e-7) # Stabilizzazione logaritmica

    return {
        'NumeroForo': numero_foro,
        'Hu_1': hu_log[0], 'Hu_2': hu_log[1], 'Hu_3': hu_log[2], 'Hu_4': hu_log[3],
        'Hu_5': hu_log[4], 'Hu_6': hu_log[5], 'Hu_7': hu_log[6],
    }

# Eseguiamo l'estrazione
hu_features_list = []
for path in tqdm(all_mask_paths, desc="Calcolo Momenti di Hu"):
    hu_feats = calculate_hu_moments(path)
    if hu_feats:
        hu_features_list.append(hu_feats)

df_unet_hu = pd.DataFrame(hu_features_list)
print("\n--- Dataset Momenti di Hu (UNet) ---")
display(df_unet_hu.head())

In [ ]:
# =============================================================================
# CELLA 5: UNIONE FINALE E SALVATAGGIO (senza geometrie umane)  ✅
# =============================================================================
print("--- Unione di UNet (geometria+Hu) con le Forze ---")

# 0) Protezione: tieni solo fori riconosciuti (>0) dalle maschere
df_unet_geometric = df_unet_geometric[df_unet_geometric["NumeroForo"].fillna(-1).astype(int) > 0]
df_unet_hu        = df_unet_hu[df_unet_hu["NumeroForo"].fillna(-1).astype(int) > 0]

# 1) UNET: merge interno tra geometria e Hu
df_unet_completo = df_unet_geometric.merge(df_unet_hu, on="NumeroForo", how="inner")

# 2) FORZE: normalizza chiave e deduplica
df_forza_maurizio = (
    df_forza_maurizio
      .assign(NumeroForo=(df_forza_maurizio["NumeroForo"]
                          .astype(str).str.extract(r"(\d+)").squeeze()))
      .dropna(subset=["NumeroForo"])
)
df_forza_maurizio["NumeroForo"] = df_forza_maurizio["NumeroForo"].astype("int64")
df_forza_maurizio = df_forza_maurizio.drop_duplicates(subset=["NumeroForo"], keep="first")

# 3) MERGE finale: tieni tutti i fori che hanno features UNet, attacca la Forza se c’è
#    (on deve essere colonna presente in ENTRAMBI i DataFrame)
df_master = df_unet_completo.merge(
    df_forza_maurizio[["NumeroForo","Forza_N"]],
    on="NumeroForo", how="left"
)

# 4) Ordina/riordina colonne e salva
df_master = df_master.sort_values("NumeroForo").reset_index(drop=True)
colonne_ordinate = [
    "NumeroForo", "Forza_N",
    "AreaDelaminata_unet", "Dmax_unet", "DF_unet", "ShapeFactor_unet",
    "Hu_1","Hu_2","Hu_3","Hu_4","Hu_5","Hu_6","Hu_7",
    "FileMaschera"
]
df_master = df_master[[c for c in colonne_ordinate if c in df_master.columns]]
output_filename = "dataset_master_finalissimo.csv"
df_master.to_csv(output_filename, index=False)

print(f"\n✅ Salvato: {output_filename}")
print(f"Righe: {len(df_master)} | Forze mancanti: {df_master['Forza_N'].isna().sum()}")

display(df_master.head(10))


Questo è un classico problema di **regressione** e di **imputazione dei dati tramite modello**. Procediamo passo dopo passo.
si passa alla parte di chiussura del dataset
1) attiva mlp per forze da geometria
2) attiva ltsm per danno

In [ ]:
#@title Ecco il codice Python che realizza esattamente questo. Utilizzeremo `pandas` per la manipolazione dei dati, `scikit-learn` per il pre-processing e la divisione dei dati, e `tensorflow/keras` per costruire e addestrare la nostra MLP.
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import re # Per estrarre il tipo di materiale dal nome del file


In [ ]:
#@title
# 1. Caricamento e Preparazione dei Dati
# --------------------------------------------
# Assicurati che il file si chiami 'dati_completi.csv' o modifica il nome
# Ho ipotizzato che il file sia nella stessa cartella dello script.
try:
    df = pd.read_csv('/content/dataset_master_finalissimo.csv') # Sostituisci con il nome del tuo file
except FileNotFoundError:
    print("Errore: File non trovato. Assicurati che il nome del file sia corretto e che si trovi nella giusta directory.")
    exit()

print("Dataset caricato con successo.")
print(f"Dimensioni del dataset: {df.shape}")

# Funzione per estrarre il tipo di materiale
def get_material_type(filename):
    # Cerca un pattern tipo "Carbon Textile XY"
    match = re.search(r'Carbon Textile \d[A-Z]', str(filename))
    if match:
        return match.group(0)
    return "Unknown" # In caso non trovi il pattern

# Crea la colonna 'TipoMateriale' che è una feature FONDAMENTALE
df['TipoMateriale'] = df['FileMaschera'].apply(get_material_type)

print("\nValori unici in 'TipoMateriale':")
print(df['TipoMateriale'].value_counts())


In [ ]:
# 2. Divisione del Dataset
# --------------------------------------------
# Dividiamo il dataframe in due: uno con le forze note (per addestrare)
# e uno con le forze mancanti (su cui predire).

df_con_forza = df[df['Forza_N'].notna()].copy()
df_senza_forza = df[df['Forza_N'].isna()].copy()

print(f"\nNumero di campioni con forza nota (per training/validation): {len(df_con_forza)}")
print(f"Numero di campioni con forza mancante (per predizione): {len(df_senza_forza)}")


In [ ]:
# 3. Definizione delle Feature e del Target
# --------------------------------------------
# Definiamo le colonne che useremo come input (features) e output (target)
# Il target è la forza, le features sono tutto il resto che descrive il danno.

features_numeriche = [
    'AreaDelaminata_unet', 'Dmax_unet', 'DF_unet', 'ShapeFactor_unet',
    'Hu_1', 'Hu_2', 'Hu_3', 'Hu_4', 'Hu_5', 'Hu_6', 'Hu_7'
]
features_categoriche = ['TipoMateriale']

# Variabili indipendenti (X) e dipendente (y)
X = df_con_forza[features_numeriche + features_categoriche]
y = df_con_forza['Forza_N']


In [ ]:
# 4. Creazione della Pipeline di Pre-processing
# --------------------------------------------
# Questa pipeline gestirà la standardizzazione delle feature numeriche
# e la codifica one-hot di quelle categoriche in modo automatico e sicuro.

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), features_numeriche),
        ('cat', OneHotEncoder(handle_unknown='ignore'), features_categoriche)
    ])

In [ ]:
# 5. Divisione del set di dati noti in Training e Validation
# -------------------------------------------------------------
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nDimensioni del set di addestramento: {X_train.shape}")
print(f"Dimensioni del set di validazione: {X_val.shape}")


**Verifica Quantitativa dell'Importanza delle Features (il test decisivo):**
    *   **Testo:** "Per quantificare oggettivamente l'importanza predittiva di ciascuna variabile, sono stati confrontati due modelli Gradient Boosting (LightGBM): un modello 'Baseline' addestrato solo su `NumeroForo` e un modello 'Completo' addestrato su tutte le features."
    <details>*   **Snippet di Codice:**
        ```python
        # Snippet 2: Confronto Modelli LGBM
        import lightgbm as lgb
        from sklearn.metrics import mean_squared_error
        import numpy as np

        df_known_force = df[df['Forza_N'].notna()]
        X = df_known_force[features]
        y = df_known_force[target]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Modello Completo
        lgbm_full = lgb.LGBMRegressor(random_state=42, verbosity=-1) # verbosity=-1 per pulire i warning
        lgbm_full.fit(X_train, y_train)
        rmse_full = np.sqrt(mean_squared_error(y_test, lgbm_full.predict(X_test)))

        # Modello Baseline
        lgbm_base = lgb.LGBMRegressor(random_state=42, verbosity=-1)
        lgbm_base.fit(X_train[['NumeroForo']], y_train)
        rmse_base = np.sqrt(mean_squared_error(y_test, lgbm_base.predict(X_test[['NumeroForo']])))

        print(f"RMSE Modello Completo: {rmse_full:.2f} N")
        print(f"RMSE Modello Baseline: {rmse_base:.2f} N")
        ```
    *   **Risultato:** Inserisci il grafico della **Feature Importance del modello completo**.
    *   **Commento:** "I risultati sono stati inequivocabili. Il modello Baseline ha ottenuto un RMSE di 6.11 N, mentre il modello Completo ha ottenuto un RMSE di 6.61 N, indicando un **peggioramento della performance del 8.13%**. Il grafico di Feature Importance (Figura X) conferma che le features geometriche del danno forniscono un contributo predittivo trascurabile una volta che l'effetto dell'usura è noto."


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

# Assumo che 'full_data.csv' sia disponibile con i tuoi dati
try:
    df = pd.read_csv('/content/dataset_master_finalissimo.csv', sep=',')
except FileNotFoundError:
    print("ERRORE: Assicurati che il file '/content/dataset_master_finalissimo.csv' sia nella stessa cartella.")
    # Fallback su dati di esempio se non trova il file
    df = pd.DataFrame({
        'NumeroForo': range(1, 11),
        'Forza_N': np.linspace(90, 110, 10),
        'AreaDelaminata_unet': np.random.rand(10) * 0.1,
        'Dmax_unet': np.random.rand(10) * 2 + 8,
        'DF_unet': np.random.rand(10) * 0.5 + 1,
        'ShapeFactor_unet': np.random.rand(10) * 0.005,
        'Hu_1': np.random.rand(10) * 2, 'Hu_2': np.random.rand(10) * 4, 'Hu_3': np.random.rand(10) * 6,
        'Hu_4': np.random.rand(10) * 6, 'Hu_5': np.random.rand(10) * 20 - 10, 'Hu_6': np.random.rand(10) * 20 - 10,
        'Hu_7': np.random.rand(10) * 20 - 10
    })

# --- PASSO 0: Preparazione ---
df['Forza_N'] = pd.to_numeric(df['Forza_N'], errors='coerce')
df_known_force = df[df['Forza_N'].notna()].copy()

features = [
    'NumeroForo',
    'AreaDelaminata_unet', 'Dmax_unet', 'DF_unet', 'ShapeFactor_unet',
    'Hu_1', 'Hu_2', 'Hu_3', 'Hu_4', 'Hu_5', 'Hu_6', 'Hu_7'
]
target = 'Forza_N'

X = df_known_force[features]
y = df_known_force[target]

# --- PASSO 1: Analisi di Correlazione ---
correlation_matrix = df_known_force[features + [target]].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f",
            annot_kws={"size": 8})
plt.title('Matrice di Correlazione tra Features e Forza', fontsize=16)
plt.show()

print("\nCorrelazione delle features con la Forza:")
print(correlation_matrix[target].sort_values(ascending=False))


# --- PASSO 2: Feature Importance con LightGBM ---

# Divisione dati
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modello B (Completo)
lgbm_full = lgb.LGBMRegressor(random_state=42)
lgbm_full.fit(X_train, y_train)

# Valutazione Modello B
preds_full = lgbm_full.predict(X_test)
rmse_full = np.sqrt(mean_squared_error(y_test, preds_full))
print(f"\nRMSE del Modello Completo (con tutte le features): {rmse_full:.4f} N")

# Visualizzazione Feature Importance
lgb.plot_importance(lgbm_full, figsize=(10, 8), title="Feature Importance (Modello Completo)")
plt.show()


# --- PASSO 3: Confronto con Modello Baseline ---

# Modello A (Baseline - solo usura)
X_train_base = X_train[['NumeroForo']]
X_test_base = X_test[['NumeroForo']]

lgbm_base = lgb.LGBMRegressor(random_state=42)
lgbm_base.fit(X_train_base, y_train)

# Valutazione Modello A
preds_base = lgbm_base.predict(X_test_base)
rmse_base = np.sqrt(mean_squared_error(y_test, preds_base))
print(f"RMSE del Modello Baseline (solo con NumeroForo): {rmse_base:.4f} N")

# --- PASSO 4: Discussione ---
improvement = (rmse_base - rmse_full) / rmse_base * 100
print(f"\nAggiungere le features del danno ha migliorato la predizione del {improvement:.2f}%.")

if improvement > 5: # Soglia arbitraria
    print("\nCONCLUSIONE: Sì, le caratteristiche della forma del danno (momenti, area, etc.)")
    print("contengono informazioni preziose per stimare la forza, anche dopo aver considerato l'usura.")
    print("Questo supporta l'idea di poter definire un 'limite di danno accettabile' basato sulla sua forma.")
else:
    print("\nCONCLUSIONE: L'effetto dominante è l'usura ('NumeroForo').")
    print("Le caratteristiche del danno aggiungono poche informazioni aggiuntive per predire la forza.")
    print("Sarà più difficile definire un limite di danno basandosi solo sulla sua forma, indipendentemente dall'usura.")

##:fine sopra preparazione dati

# **Sott::o inizio settaggio modello**!!:

#modello MLP usato all'inizio, ma mancano picchi, sembra seguire medie, usaimo un modello diverso che è più centrato sulla relazione fra metriche per capire le correlazioni fra forza e danno

In [ ]:
# 6. Costruzione del Modello MLP (Multi-Layer Perceptron)
# --------------------------------------------------------
# Creiamo una pipeline che prima processa i dati e poi li passa al modello
# Definiamo un'architettura MLP semplice ma efficace.

# Trasformiamo i dati di training e validation usando il preprocessor
X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)

# Definiamo il modello Keras
model = tf.keras.Sequential([
    # Il numero di neuroni nel primo strato è basato sulla forma dei dati processati
    tf.keras.layers.Input(shape=(X_train_processed.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2), # Dropout per ridurre l'overfitting
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1) # Strato di output con 1 neurone per la regressione
])

# Compiliamo il modello
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error', # Ottima loss per la regressione
    metrics=['mean_absolute_error'] # Metrica più interpretabile (errore medio in N)
)

model.summary()

# Aggiungiamo un early stopping per fermare l'addestramento quando il modello smette di migliorare
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=15, restore_best_weights=True
)


In [ ]:
# 7. Addestramento del Modello
# --------------------------------------------------------
print("\nInizio addestramento MLP...")
history = model.fit(
    X_train_processed, y_train,
    epochs=200, # Numero massimo di epoche
    validation_data=(X_val_processed, y_val),
    callbacks=[early_stopping],
    verbose=1
)
print("Addestramento completato.")


In [ ]:
# 8. Valutazione del Modello
# --------------------------------------------------------
# Visualizziamo le curve di apprendimento per assicurarci che non ci sia overfitting
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss del Modello')
plt.xlabel('Epoca')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['mean_absolute_error'], label='Training MAE')
plt.plot(history.history['val_mean_absolute_error'], label='Validation MAE')
plt.title('Errore Assoluto Medio (MAE)')
plt.xlabel('Epoca')
plt.ylabel('Forza (N)')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 9. Predizione delle Forze Mancanti
# --------------------------------------------------------
# Usiamo il modello addestrato per predire i valori sul set df_senza_forza
X_da_predire = df_senza_forza[features_numeriche + features_categoriche]

# Processiamo i dati da predire con lo stesso preprocessor
X_da_predire_processed = preprocessor.transform(X_da_predire)

# Eseguiamo la predizione
forze_predette = model.predict(X_da_predire_processed).flatten() # .flatten() per avere un array 1D


In [ ]:
# 10. Creazione del DataFrame Finale
# ---------------------------------------------------------
# Riempiamo i valori NaN nel dataframe originale con le nostre predizioni
df_finale = df.copy()
df_finale.loc[df_finale['Forza_N'].isna(), 'Forza_N'] = forze_predette

# Visualizziamo alcuni dei valori imputati per controllo
print("\nDataFrame originale con forze mancanti (prime 5 righe affette):")
print(df[df['Forza_N'].isna()].head())

print("\nDataFrame finale con forze imputate (stesse 5 righe):")
print(df_finale.loc[df_senza_forza.index].head())

# Salviamo il nuovo CSV completo
output_filename = 'dati_con_forze_imputate.csv'
df_finale.to_csv(output_filename, index=False)
print(f"\nDataset completo salvato come '{output_filename}'")

In [ ]:
##NEW
# =============================================================================
# CELLA 5: STUDIO ESPLORATIVO + ESPORTAZIONE/DOWNLOAD
# =============================================================================
# Requisiti: df_unet_geometric, df_unet_hu già creati (CELLE 3–4)
# Opzionale (se hai già fatto il merge forza altrove): df_forza_maurizio

import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt
from google.colab import files
from pathlib import Path

plt.close('all')
sns.set(context="notebook", style="whitegrid")

# ---- 1) Build preview dataframe (join geom + Hu; attacca Forza se disponibile) ----
df_unet = pd.merge(df_unet_geometric, df_unet_hu, on="NumeroForo", how="inner")

if "df_forza_maurizio" in globals() and isinstance(df_forza_maurizio, pd.DataFrame):
    tmp_forze = df_forza_maurizio.copy()
    if "NumeroForo" in tmp_forze.columns:
        tmp_forze["NumeroForo"] = (
            tmp_forze["NumeroForo"].astype(str).str.extract(r"(\d+)").astype(float)
        )
        tmp_forze["NumeroForo"] = tmp_forze["NumeroForo"].astype("Int64")
    df_master_preview = df_unet.merge(
        tmp_forze[["NumeroForo","Forza_N"]].drop_duplicates("NumeroForo"),
        on="NumeroForo", how="left"
    )
else:
    df_master_preview = df_unet.copy()
    if "Forza_N" not in df_master_preview.columns:
        df_master_preview["Forza_N"] = np.nan  # slot per coerenza viste

# Colonne numeriche principali da studiare (adatta liberamente)
num_cols = [c for c in [
    "AreaDelaminata_unet","Dmax_unet","DF_unet","ShapeFactor_unet",
    "Hu_1","Hu_2","Hu_3","Hu_4","Hu_5","Hu_6","Hu_7","Forza_N"
] if c in df_master_preview.columns]

print(f"Rows in preview: {len(df_master_preview)} | numeric cols: {len(num_cols)}")

# ---- 2) Distributions (hist+KDE) ----
for c in num_cols:
    fig = plt.figure(figsize=(6,4))
    ax = sns.histplot(df_master_preview[c].dropna(), kde=True)
    ax.set_title(f"Distribution — {c}")
    ax.set_xlabel(c); ax.set_ylabel("count")
    plt.show()

# ---- 3) 2-D pairwise (quick) ----
# Hint: subset per velocità se vuoi: vars=num_cols[:6]
if len(num_cols) >= 2:
    g = sns.pairplot(df_master_preview[num_cols].dropna(), corner=True, diag_kind="hist")
    g.fig.suptitle("Pairwise (subset numeric)", y=1.02)
    plt.show()

# ---- 4) Correlation heatmap ----
if len(num_cols) >= 2:
    corr = df_master_preview[num_cols].corr(numeric_only=True)
    plt.figure(figsize=(8,6))
    ax = sns.heatmap(corr, annot=False, cmap="viridis", square=True)
    ax.set_title("Correlation (numeric)")
    plt.tight_layout(); plt.show()

# ---- 5) “Time series” semplice indicizzata per foro (giusto per scorrere valori) ----
if "Forza_N" in df_master_preview.columns:
    df_plot = df_master_preview.sort_values("NumeroForo")
    plt.figure(figsize=(8,3.5))
    plt.plot(df_plot["NumeroForo"], df_plot["Forza_N"], marker=".", linestyle="-")
    plt.title("Forza_N vs NumeroForo")
    plt.xlabel("NumeroForo"); plt.ylabel("Forza_N")
    plt.tight_layout(); plt.show()

# ---- 6) Salvataggi + download ----
OUT_DIR = Path("/content"); OUT_DIR.mkdir(exist_ok=True, parents=True)
p_geom = OUT_DIR/"unet_geometria.csv"
p_hu   = OUT_DIR/"unet_hu.csv"
p_prev = OUT_DIR/"dataset_master_preview.csv"

df_unet_geometric.to_csv(p_geom, index=False)
df_unet_hu.to_csv(p_hu, index=False)
df_master_preview.to_csv(p_prev, index=False)

print("Saved:")
print(" -", p_geom)
print(" -", p_hu)
print(" -", p_prev)

# trigger download (Colab)
for p in [p_geom, p_hu, p_prev]:
    try:
        files.download(str(p))
    except Exception as e:
        print(f"(Info) Download programmatico non riuscito per {p.name}. Scaricalo dal pannello Files. Dettagli: {e}")

# ---- 7) (Opzionale) Report HTML EDA con ydata-profiling ----
# NB: generare il report può richiedere un po' di tempo per molte colonne/righe
try:
    %pip -q install ydata-profiling
    from ydata_profiling import ProfileReport
    profile = ProfileReport(df_master_preview[num_cols + ["NumeroForo"]], title="UNet + Forze — Profiling")
    html_path = OUT_DIR/"eda_report_unet_forze.html"
    profile.to_file(html_path)
    print("EDA report:", html_path)
    try:
        files.download(str(html_path))
    except Exception as e:
        print(f"(Info) Download HTML non auto-avviato: {e}")
except Exception as e:
    print("(Opzionale) ydata-profiling non installato/errore:", e)


In [ ]:
### Analisi Grafica della Coerenza dei Valori di Forza Imputati
'''
Per questa analisi, utilizzerò il file originale e quello nuovo, `/content/dati_con_forze_imputate.csv.csv`.

```python'''
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Impostazioni grafiche per una migliore leggibilità
sns.set_theme(style="whitegrid")

# 1. Caricamento dei due dataset per il confronto
# ----------------------------------------------------
try:
    df_original = pd.read_csv('/content/dataset_master_finalissimo.csv')
    df_imputato = pd.read_csv('/content/dati_con_forze_imputate.csv')
except FileNotFoundError:
    print("Assicurati che entrambi i file '/content/dataset_master_finalissimo.csv' e '/content/dati_con_forze_imputate.csv.csv' siano presenti.")
    exit()

# Aggiungiamo un flag per distinguere i dati originali, imputati e reali
df_imputato['Stato'] = 'Reale'
# Identifichiamo gli indici dove la forza era mancante nel file originale
indici_mancanti = df_original[df_original['Forza_N'].isna()].index
df_imputato.loc[indici_mancanti, 'Stato'] = 'Imputato'

print("Dati caricati e pronti per la visualizzazione.")

# 2. Grafico 1: Andamento Generale della Forza
# ----------------------------------------------------
# Questo grafico ci mostra la sequenza completa delle forze, evidenziando i punti che abbiamo predetto.
# Ci aspettiamo che i punti rossi (imputati) si inseriscano in modo fluido nell'andamento generale.

plt.figure(figsize=(18, 8))
sns.scatterplot(
    data=df_imputato,
    x='NumeroForo',
    y='Forza_N',
    hue='Stato',
    palette={'Reale': 'blue', 'Imputato': 'red'},
    s=20, # Dimensione dei punti
    alpha=0.7
)
plt.title('Andamento Generale della Forza (Valori Reali vs Imputati)', fontsize=16)
plt.xlabel('Numero del Foro (Sequenza Esperimento)', fontsize=12)
plt.ylabel('Forza (N)', fontsize=12)
plt.legend(title='Stato del Dato')
plt.show()

# 3. Grafico 2: Confronto delle Distribuzioni (Violin Plot)
# -----------------------------------------------------------------
# Questo grafico è FONDAMENTALE. Mostra la distribuzione dei valori di forza per ogni tipo di materiale.
# Vogliamo vedere se la distribuzione dei valori imputati (in rosso) si sovrappone bene a quella
# dei valori reali (in blu) all'interno dello stesso gruppo di materiale.

# Ordiniamo i materiali per una visualizzazione logica
material_order = sorted(df_imputato['TipoMateriale'].unique())

plt.figure(figsize=(18, 9))
sns.violinplot(
    data=df_imputato,
    x='TipoMateriale',
    y='Forza_N',
    hue='Stato',
    split=True, # Divide il violino a metà per un confronto diretto
    inner='quart', # Mostra i quartili all'interno
    palette={'Reale': 'skyblue', 'Imputato': 'salmon'},
    order=material_order
)
plt.title('Distribuzione delle Forze per Tipo di Materiale', fontsize=16)
plt.xlabel('Tipo di Materiale', fontsize=12)
plt.ylabel('Forza (N)', fontsize=12)
plt.xticks(rotation=45)
plt.legend(title='Stato del Dato', loc='upper left')
plt.tight_layout()
plt.show()

# 4. Grafico 3: Relazione Forza vs. Area Delaminata
# -----------------------------------------------------------------
# Verifichiamo se i punti imputati seguono la stessa "nuvola" di punti dei dati reali quando
# plottiamo la forza contro una delle feature più importanti.

plt.figure(figsize=(18, 8))
sns.scatterplot(
    data=df_imputato,
    x='AreaDelaminata_unet',
    y='Forza_N',
    hue='Stato',
    style='TipoMateriale', # Usiamo stili diversi per ogni materiale
    palette={'Reale': 'black', 'Imputato': 'red'},
    s=50,
    alpha=0.8
)
plt.title('Correlazione Forza vs. Area Delaminata (Valori Reali vs Imputati)', fontsize=16)
plt.xlabel('Area Delaminata (mm²)', fontsize=12)
plt.ylabel('Forza (N)', fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()
plt.show()